<h3>Глобальные переменные</h3>

In [3]:
transformer_model_path = 'roberta-base'
USE_CUDA = True

<h3>Векторизуем словарь Meddra</h3>

In [2]:
from sentence_transformers import SentenceTransformer
from vectorization import ConceptVectorizer

#роберта с mean pooling
model = SentenceTransformer('all-distilroberta-v1')

In [5]:
#альтернатива
from vectorization import ConceptVectorizer

CV = ConceptVectorizer(transformer_model_path, '../../Data/External/pt_en.asc', \
                             use_concept_less=False, use_model=False, use_cuda=USE_CUDA)
CV_with_conceptless = ConceptVectorizer(transformer_model_path, '../../Data/External/pt_en.asc', \
                             use_concept_less=True, use_model=False, use_cuda=USE_CUDA)

Расчет вложений словаря

In [ ]:
CV.thesaurus_embeddings = model.encode(list(CV.meddra_term_to_meddra_code.keys()))
CV.normalization_mode = 'mean_pooling'

Сохранение рассчитанных векторов концептов

In [9]:
import torch

torch.save(CV.thesaurus_embeddings, './Model_weights/distilroberta_thesaurus_embeddings_en.pt')

Загрузка готовых вложений (если они есть). Загружать вложения после fit_transform() не надо

In [6]:
import torch

CV.thesaurus_embeddings = torch.load('./Model_weights/distilroberta_thesaurus_embeddings_en.pt')
CV.normalization_mode = 'mean_pooling'

In [7]:
CV.thesaurus_embeddings.shape

(25077, 768)

<h3>CADEC</h3>

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

data_dir = '../../Data/Raw/CADEC/cadec_table_grartem.csv'

df = pd.read_csv(data_dir)

In [9]:
df = df[df['meddra']!='CONCEPT_LESS']
df['meddra'] =  df['meddra'].apply(lambda x: str(x).split('|')[0])
mask = df['meddra'].apply(lambda x: True if x in CV.meddra_codes else False)
df = df[mask]


all_pt_ids = df['meddra'].to_list()
all_pt_phrases = df['term'].to_list()
#train_phrases, test_phrases, train_concepts, test_concepts = train_test_split(all_pt_phrases, all_pt_ids, test_size=0.33, random_state=42)

In [10]:
from sklearn.model_selection import KFold

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
folds = []

terms = df['term'].to_numpy()
concepts = df['meddra'].to_numpy()

for train_index, test_index in kf.split(df['term']):
    train_phrases = terms[train_index]
    test_phrases = terms[test_index]
    train_concepts = concepts[train_index]
    test_concepts = concepts[test_index]
    folds.append([[train_phrases, train_concepts], [test_phrases, test_concepts]])

In [14]:
fold_num = 1
train_phrases, train_concepts = folds[fold_num][0][0], folds[fold_num][0][1]
test_phrases, test_concepts = folds[fold_num][1][0], folds[fold_num][1][1]

print('Всего фраз в трейне: %s'%len(train_phrases))
print('Всего фраз в тесте: %s'%len(test_phrases))

print('Уникальных фраз в трейне: %s'%len(set(train_phrases)))
print('Уникальных фраз в тесте: %s'%len(set(test_phrases)))

#Посмотрим на статистику разбиения
print('%s концептов не входящих либо в трейн, либо в тест'%len(set.union(set(train_concepts), set(test_concepts)) - set.intersection(set(test_concepts), set(train_concepts))))
print('%s концептов, которые есть в тесте, но нет в трейне'%len(set(test_concepts) - set(train_concepts)))
print('%s концептов, которые есть в трейне, но нет в тесте'%len(set(train_concepts) - set(test_concepts)))

Всего фраз в трейне: 4773
Всего фраз в тесте: 1194
Уникальных фраз в трейне: 2789
Уникальных фраз в тесте: 848
254 концептов не входящих либо в трейн, либо в тест
30 концептов, которые есть в тесте, но нет в трейне
224 концептов, которые есть в трейне, но нет в тесте


---

<h2>Обучение</h2>

<h4>Настройка глобальных переменных для большей детерменированности</h4>

In [15]:
import os
import numpy as np

os.environ["CUBLAS_WORKSPACE_CONFIG"]=":16:8"
torch.use_deterministic_algorithms(mode=False)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False

<h3>Кроссвалидация</h3>

In [12]:
from models import CADEC_SoTa
from dataset import MedNormDataset
import torch.optim as optim
import torch.nn as nn
import torch
from evaluator import Evaluator
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score

#Гиперпараметры
batch_size=16
epochs = 25

for fold_num in range(0, 1):
    print('exp: %s\n'%fold_num)
    train_phrases, train_concepts = list(folds[fold_num][0][0]), list(folds[fold_num][0][1])
    test_phrases, test_concepts = list(folds[fold_num][1][0]), list(folds[fold_num][1][1])

    ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=True)
    ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=True)

    net = CADEC_SoTa(transformer_model_path, torch.tensor(CV.thesaurus_embeddings))
    #перемещение модели на вычислительные мощности, либо cpu, либо cuda
    device = 'cuda' if USE_CUDA else 'cpu'
    net.to(device)
    print('Net loaded')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0001) #, cycle_momentum=True

    scaler = torch.cuda.amp.GradScaler()
    evaluator = Evaluator(train_concepts, test_concepts)
    
    trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
    testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)


    initial_loss = None

    for epoch in range(1, epochs+1):
        #обучение модели в эпохе
        net.train()
        with tqdm(trainloader, unit="batch") as tepoch:
            for data in tepoch:

                tepoch.set_description(f"Epoch {epoch}")

                inputs = data['tokenized_phrases']
                labels = data['one_hot_labels']

                optimizer.zero_grad()
                if USE_CUDA=='cuda':
                    with torch.cuda.amp.autocast():
                        outputs = net(inputs)['output']
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                if initial_loss is None:
                    initial_loss = loss.item()
                tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'][0])
    evaluator.get_all_f1_scores(real_answers, model_answers)
print('Finished Training')
torch.save(net,'./Model_weights/CADEC_exp_%s.pt'%fold_num)

exp: 0



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Net loaded


100%|██████████| 1194/1194 [00:12<00:00, 94.45batch/s]


All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.894472  0.636641  0.910256  0.701305      0.125  0.073171

Finished Training


In [17]:
from models import CADEC_SoTa
from dataset import MedNormDataset
import torch.optim as optim
import torch.nn as nn
import torch
from evaluator import Evaluator
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score

#Гиперпараметры
batch_size=16
epochs = 25

for fold_num in range(1,6):
    print('exp: %s\n'%fold_num)
    train_phrases, train_concepts = list(folds[fold_num][0][0]), list(folds[fold_num][0][1])
    test_phrases, test_concepts = list(folds[fold_num][1][0]), list(folds[fold_num][1][1])

    ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=True)
    ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=True)

    net = CADEC_SoTa(transformer_model_path, torch.tensor(CV.thesaurus_embeddings))
    #перемещение модели на вычислительные мощности, либо cpu, либо cuda
    device = 'cuda' if USE_CUDA else 'cpu'
    net.to(device)
    print('Net loaded')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0001) #, cycle_momentum=True

    scaler = torch.cuda.amp.GradScaler()
    evaluator = Evaluator(train_concepts, test_concepts)
    
    trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
    testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)


    initial_loss = None

    for epoch in range(1, epochs+1):
        #обучение модели в эпохе
        net.train()
        with tqdm(trainloader, unit="batch") as tepoch:
            for data in tepoch:

                tepoch.set_description(f"Epoch {epoch}")

                inputs = data['tokenized_phrases']
                labels = data['one_hot_labels']

                optimizer.zero_grad()
                if USE_CUDA=='cuda':
                    with torch.cuda.amp.autocast():
                        outputs = net(inputs)['output']
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                if initial_loss is None:
                    initial_loss = loss.item()
                tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
        #расчет точностей модели после эпохи
        net.eval()
        model_answers=[]
        real_answers=[]
        with tqdm(testloader, unit="batch") as eval_process:
            for data in eval_process:

                inputs = data['tokenized_phrases']

                with torch.no_grad():
                    outputs_dict = net(inputs)
                    pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]


                model_answers.append(pred_meddra_code)
                real_answers.append(data['label_codes'][0])
        evaluator.get_all_f1_scores(real_answers, model_answers)
    print('Finished Training')
    torch.save(net,'./Model_weights/CADEC_exp_%s.pt'%fold_num)

exp: 1



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Net loaded


100%|██████████| 1194/1194 [00:13<00:00, 89.94batch/s] 


All             In VOC           Out of VOC          
       f1 micro  f1 macro f1 micro  f1 macro   f1 micro  f1 macro
score  0.657454  0.201744  0.67528  0.229217   0.030303  0.019608

...

All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.851635  0.514013  0.886264  0.666352       0.06  0.035294

100%|██████████| 1193/1193 [00:13<00:00, 88.28batch/s] 


All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.851635  0.517791  0.886264  0.668846       0.06  0.035294

Finished Training
exp: 5



IndexError: list index out of range

In [14]:
import numpy as np

f1_scores = [0.894472, 0.858459, 0.860017, 0.867561, 0.851635]
print('f1: %s +- %s'%(np.round(np.mean(f1_scores), 4), np.round(np.std(f1_scores), 4)))

f1: 0.8664 +- 0.0149


In [15]:
import numpy as np

f1_scores = [0.517791, 0.551561, 0.547865, 0.527303, 0.636641]
print('f1: %s +- %s'%(np.round(np.mean(f1_scores), 4), np.round(np.std(f1_scores), 4)))

f1: 0.5562 +- 0.0421


<h4>Импорт модели</h4>

In [ ]:
net = CADEC_SoTa(transformer_model_path, torch.tensor(CV.thesaurus_embeddings))
#перемещение модели на вычислительные мощности, либо cpu, либо cuda
device = 'cuda' if USE_CUDA else 'cpu'
net.to(device)
print('Net loaded')

<h4>Инициализация доп. объектов для обучения</h4>

In [34]:
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np

from torch.optim.lr_scheduler import CyclicLR
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001) #, cycle_momentum=True

scaler = torch.cuda.amp.GradScaler()
evaluator = Evaluator(train_concepts, test_concepts)

<h4>Цикл обучения</h4>

In [35]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np



#Гиперпараметры
batch_size=16
epochs = 50
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)


initial_loss = None

for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if USE_CUDA=='cuda':
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'][0])
    evaluator.get_all_f1_scores(real_answers, model_answers)
print('Finished Training')
torch.save(net,'./Model_weights/rubert_additional_corp_freeze_layers_%s_exp_17.pt'%epoch)

100%|██████████| 3787/3787 [00:39<00:00, 95.63batch/s] 


All              In VOC           Out of VOC          
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.597043  0.177648  0.621208  0.199196   0.149485  0.090458

100%|██████████| 3787/3787 [00:39<00:00, 95.16batch/s] 


All              In VOC           Out of VOC          
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.621336  0.187186  0.647092  0.215558    0.14433  0.087673

100%|██████████| 3787/3787 [00:38<00:00, 97.60batch/s] 


...

100%|██████████| 3787/3787 [00:38<00:00, 97.19batch/s] 


All             In VOC           Out of VOC          
   f1 micro  f1 macro f1 micro  f1 macro   f1 micro  f1 macro
0  0.733034  0.267256  0.76816  0.345812   0.082474  0.053866

100%|██████████| 3787/3787 [00:40<00:00, 94.30batch/s] 


All              In VOC          Out of VOC          
   f1 micro  f1 macro  f1 micro f1 macro   f1 micro  f1 macro
0  0.736731  0.270444  0.772613  0.35222   0.072165  0.050562

Finished Training


<h4>не помню</h4>

In [16]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np



#Гиперпараметры
batch_size=64
epochs = 50
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)


initial_loss = None

for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if USE_CUDA=='cuda':
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'][0])
    evaluator.get_all_f1_scores(real_answers, model_answers)
print('Finished Training')
torch.save(net,'./Model_weights/rubert_additional_corp_%s_exp_16.pt'%epoch)

100%|██████████| 3787/3787 [00:40<00:00, 92.89batch/s]


All              In VOC           Out of VOC          
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.348561  0.059877  0.363206  0.063241    0.07732  0.042541

100%|██████████| 3787/3787 [00:39<00:00, 95.15batch/s] 


All              In VOC           Out of VOC         
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro f1 macro
0  0.517824  0.096982  0.540495  0.105915   0.097938  0.05716

100%|██████████| 3787/3787 [00:45<00:00, 83.57batch/s]


...

100%|██████████| 3787/3787 [00:42<00:00, 89.49batch/s] 


All              In VOC           Out of VOC         
  f1 micro  f1 macro  f1 micro  f1 macro   f1 micro f1 macro
0   0.7399  0.246421  0.767882  0.280983   0.221649  0.12538

100%|██████████| 3787/3787 [00:41<00:00, 91.59batch/s]


All              In VOC           Out of VOC          
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.742012  0.236418  0.770109  0.267377   0.221649  0.122034

Finished Training


<h3>50 эпох на доп. корпусе v2 с RDRS</h3>

In [19]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np



#Гиперпараметры
batch_size=64
epochs = 50
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)


initial_loss = None

for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if USE_CUDA=='cuda':
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'][0])
    evaluator.get_all_f1_scores(real_answers, model_answers)
print('Finished Training')
torch.save(net,'./Model_weights/rubert_additional_corp_v3_%s.pt'%epoch)

100%|██████████| 3787/3787 [00:40<00:00, 93.40batch/s]


All             In VOC           Out of VOC          
   f1 micro  f1 macro f1 micro  f1 macro   f1 micro  f1 macro
0  0.342752  0.061241  0.35764  0.064874    0.06701  0.040636

100%|██████████| 3787/3787 [00:43<00:00, 86.97batch/s]


All              In VOC           Out of VOC          
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.494587  0.100847  0.515168  0.106615   0.113402  0.067633

100%|██████████| 3787/3787 [00:43<00:00, 87.24batch/s]


...

100%|██████████| 3787/3787 [00:39<00:00, 96.19batch/s] 


All              In VOC           Out of VOC          
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.745181  0.248322  0.775675  0.290498   0.180412  0.104694

100%|██████████| 3787/3787 [00:39<00:00, 96.14batch/s] 


All             In VOC           Out of VOC         
   f1 micro f1 macro  f1 micro  f1 macro   f1 micro f1 macro
0  0.745181  0.24042  0.775953  0.283091   0.175258  0.10323

Finished Training


<h2>Подбор гиперпараметров</h2>

In [ ]:
from parameters_optimization import CADEC_SoTa_Optimizer

training_param_optimizer = CADEC_SoTa_Optimizer(net, ds_train, ds_test, 
                                                hyperparam_space={'lr': (0.00001, 0.001), 'batch_size': (16, 32, 64)},
                                                verbose=True)
best_hyperparams = training_param_optimizer.optimize()
print(best_hyperparams)

In [ ]:
'5. f1: 0.4008, lr: 0.0002819, bs: 32'

In [ ]:
'f1: 0.4563, lr: 0.0003526, bs: 16'

In [12]:
import torch
from models import CADEC_SoTa

net = torch.load('./Model_weights/rubert_only_additional_corp_v2_30ep.pt') 

<h3>Процесс обучения и рассчета точностей rubert с доп. корпусом и классом Evaluator</h3>

In [15]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np



#Гиперпараметры
batch_size=16
epochs = 50
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

evaluator = Evaluator(train_concepts, test_concepts_without_conceptless)

initial_loss = None
best_f1=0
for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if USE_CUDA=='cuda':
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'][0])
    evaluator.get_all_f1_scores(real_answers, model_answers)
print('Finished Training')
torch.save(net,'./Model_weights/rubert_additional_corp_v2_%s.pt'%epoch)

100%|██████████| 3787/3787 [00:44<00:00, 85.25batch/s]


All              In VOC          Out of VOC          
  f1 micro  f1 macro  f1 micro f1 macro   f1 micro  f1 macro
0   0.7061  0.239987  0.735597   0.2816   0.159794  0.101421

100%|██████████| 3787/3787 [00:42<00:00, 89.00batch/s]


All              In VOC           Out of VOC          
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.730922  0.259726  0.761481  0.314257   0.164948  0.097863

100%|██████████| 3787/3787 [00:42<00:00, 90.04batch/s]


...

100%|██████████| 3787/3787 [00:42<00:00, 88.42batch/s]


All             In VOC           Out of VOC          
  f1 micro f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.77502  0.32381  0.811856  0.434133   0.092784  0.057126

100%|██████████| 3787/3787 [00:42<00:00, 89.55batch/s]


All              In VOC           Out of VOC          
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.775812  0.325669  0.812691  0.435375   0.092784  0.056725

Finished Training


In [25]:
net.eval()
model_answers=[]
real_answers=[]
evaluator = Evaluator(train_concepts, test_concepts_without_conceptless)
with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        inputs = data['tokenized_phrases']

        with torch.no_grad():
            outputs_dict = net(inputs)
            pred_meddra_code = CV_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'][0])
evaluator.get_all_f1_scores(real_answers, model_answers)

100%|██████████| 3787/3787 [00:42<00:00, 89.50batch/s]


All              In VOC          Out of VOC          
  f1 micro  f1 macro  f1 micro f1 macro   f1 micro  f1 macro
0   0.7061  0.239987  0.735597   0.2816   0.159794  0.101421

In [24]:
evaluator.in_voc_idx

array([], dtype=int64)

<h3>Процесс обучения и рассчета точностей на доп. корпусе</h3>

In [17]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score

#Гиперпараметры
batch_size=64#16
epochs = 30
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

net.train()
initial_loss = None
best_f1=0
for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if device=='cuda':
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'])
    #f1 = f1_score(real_answers, model_answers, average='micro')
    
    #if f1>best_f1:
    #    best_f1=f1
    #    torch.save(net,'./Model_weights/xlmroberta_best_%s.pt'%epoch)
    #if epoch==9:
    #    torch.save(net,'./Model_weights/xlmroberta_%s.pt'%epoch)
    
    f1 = f1_score(real_answers, model_answers, average='micro')
    print('%s micro'%f1)
    f1 = f1_score(real_answers, model_answers, average='macro')
    print('%s macro'%f1)
            
print('Finished Training')
torch.save(net,'./Model_weights/rubert_only_additional_corp_v2%s.pt'%epoch)

100%|██████████| 3787/3787 [00:40<00:00, 94.32batch/s] 


0.12278848692896752 micro
0.054613601851199206 macro


100%|██████████| 3787/3787 [00:38<00:00, 97.19batch/s] 


0.1938209664642197 micro
0.06721648734675416 macro


100%|██████████| 3787/3787 [00:41<00:00, 91.63batch/s] 


...

0.4779508846052284 micro
0.15678419166189123 macro


100%|██████████| 3787/3787 [00:40<00:00, 93.85batch/s] 


0.47927119091629256 micro
0.1571352333250121 macro


100%|██████████| 3787/3787 [00:39<00:00, 95.14batch/s] 


0.4708212305254819 micro
0.15236921283847657 macro
Finished Training


<h2>Анализ ошибок rubert</h2>

In [94]:
from tabulate import tabulate
import pandas as pd

class Evaluator:
    '''Расчет точностей и работа с индексами In and Out of VOC'''
    def __init__(self, train_conepts, test_concepts):
        self.train_conepts, self.test_concepts = np.array(train_conepts), np.array(test_concepts)
        self.get_in_out_of_voc_idx()
        
    def get_in_out_of_voc_idx(self):
        mask_out_of_voc = np.in1d(self.test_concepts, self.train_conepts) #маска элементов первого array, которые есть во втором array
        self.in_voc_idx = np.nonzero(mask_out_of_voc)[0] 
        self.out_of_voc_idx = np.nonzero(~mask_out_of_voc)[0]
        
    def get_f1_score(self, y_true, y_pred, average, idx='All'):
        if idx=='All':
            return f1_score(y_true, y_pred, average=average)
        elif idx=='In VOC':
            return f1_score(y_true[self.in_voc_idx], y_pred[self.in_voc_idx], average=average)
        elif idx=='Out of VOC':
            return f1_score(y_true[self.out_of_voc_idx], y_pred[self.out_of_voc_idx], average=average)
        
    def get_all_f1_scores(self, y_true, y_pred, pretty_print=True):
        columns_tuples = [('All', 'f1 micro'), 
                     ('All', 'f1 macro'),
                                           
                     ('In VOC', 'f1 micro'), 
                     ('In VOC', 'f1 macro'),
                                           
                     ('Out of VOC', 'f1 micro'), 
                     ('Out of VOC', 'f1 macro')]
        all_f1_scores = []
        for cfg_row in columns_tuples:
            all_f1_scores.append(self.get_f1_score(np.array(y_true), np.array(y_pred), 
                                                   average=cfg_row[1][3:], idx=cfg_row[0]))
        all_f1_scores = np.array(all_f1_scores).reshape((1,6))
        col_index = pd.MultiIndex.from_tuples(columns_tuples)
        self.f1_scores_df = pd.DataFrame(all_f1_scores, columns=col_index)
        if pretty_print:
            display(self.f1_scores_df) #, headers='keys', tablefmt='psql')

In [95]:
evaluator = Evaluator(train_concepts, test_concepts_without_conceptless)

In [97]:
net = torch.load('./Model_weights/rubert_50_exp_11.pt')

In [103]:
net.eval()
model_answers=[]
real_answers=[]
phrases = []
with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        inputs = data['tokenized_phrases']

        with torch.no_grad():
            outputs_dict = net(inputs)
            pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'][0])
        phrases.append(data['phrases'][0])
    evaluator.get_all_f1_scores(real_answers, model_answers)

100%|██████████| 3787/3787 [00:42<00:00, 88.59batch/s] 


All              In VOC           Out of VOC          
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.762345  0.293459  0.801837  0.421247   0.030928  0.015411

In [108]:
import pandas as pd

model_df_out = pd.DataFrame({'true': model_answers, 'pred': real_answers, 'phrases': phrases, 'test_ids': test_ids})
model_df_out['true_terms'] = model_df_out['true'].apply(lambda x: CV_train.meddra_code_to_meddra_term[x])
model_df_out['pred_terms'] = model_df_out['pred'].apply(lambda x: CV_train.meddra_code_to_meddra_term[x])
model_df_out['is wrong'] = model_df_out['pred'] != model_df_out['true']
model_df_out[model_df_out['is wrong']].to_csv('rubert_50_epochs_out.csv', index=False)

In [106]:
print(len(test_ids)) 

4006


In [90]:
len(real_answers)

3787

In [91]:
len(evaluator.test_concepts)

4006

<h3>Процесс обучения и рассчета точностей rubert</h3>

In [15]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score

#Гиперпараметры
batch_size=64#16
epochs = 50
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

net.train()
initial_loss = None
best_f1=0
for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if device=='cuda':
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'])
    f1 = f1_score(real_answers, model_answers, average='micro')
    #if f1>best_f1:
    #    best_f1=f1
    #    torch.save(net,'./Model_weights/xlmroberta_best_%s.pt'%epoch)
    #if epoch==9:
    #    torch.save(net,'./Model_weights/xlmroberta_%s.pt'%epoch)
    f1 = f1_score(real_answers, model_answers, average='micro')
    print('%s micro'%f1)
    f1 = f1_score(real_answers, model_answers, average='macro')
    print('%s macro'%f1)
            
print('Finished Training')
torch.save(net,'./Model_weights/rubert_%s.pt'%epoch)

100%|██████████| 3787/3787 [00:44<00:00, 85.89batch/s] 


0.5901769210456826 micro
0.10543854161409019 macro


100%|██████████| 3787/3787 [00:39<00:00, 96.16batch/s] 


0.646686031159229 micro
0.13236941797643076 macro


100%|██████████| 3787/3787 [00:39<00:00, 97.00batch/s] 


...

0.7618167414840243 micro
0.2914558917760406 macro


100%|██████████| 3787/3787 [00:39<00:00, 97.07batch/s] 


0.7602323739107472 micro
0.2881593763156088 macro


100%|██████████| 3787/3787 [00:38<00:00, 98.66batch/s] 


0.76234486400845 micro
0.2934589715874881 macro
Finished Training


<h3>Анализ ошибок RuBERT</h3>

In [83]:
Evaluator(, test_concepts_without_conceptless)

['10036898',
 '10036898',
 'CONCEPT_LESS',
 '10036898',
 '10036898',
 '10063420',
 '10037430',
 '10042209',
 '10062519',
 '10029216',
 '10022998',
 '10011224',
 '10039101',
 '10005911',
 '10022437',
 '10022998',
 '10002855',
 '10042209',
 '10011224',
 '10028813',
 '10005911',
 '10039101',
 '10011224',
 '10037660',
 '10016766',
 '10013082',
 '10017367',
 '10020751',
 '10015150',
 '10013082',
 '10033616',
 '10000496',
 '10040007',
 '10039101',
 '10022437',
 '10062519',
 '10042209',
 '10042209',
 '10039101',
 '10039101',
 '10036898',
 '10033371',
 '10003239',
 '10003051',
 '10061218',
 '10023204',
 '10011224',
 '10039101',
 '10039101',
 '10039101',
 '10011224',
 '10011224',
 '10062519',
 '10029412',
 '10027783',
 '10062519',
 '10029216',
 '10068319',
 '10021425',
 '10012735',
 '10047700',
 '10011224',
 '10014551',
 '10022998',
 '10061284',
 '10022437',
 'CONCEPT_LESS',
 '10000060',
 '10061422',
 '10016766',
 '10039740',
 '10022998',
 '10010774',
 '10005750',
 '10028735',
 '10028735',
 '10

In [ ]:
#testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)
net.eval()
model_answers=[]
real_answers=[]
with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        inputs = data['tokenized_phrases']

        with torch.no_grad():
            outputs_dict = net(inputs)
            pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'][0])
        
    

In [26]:
f1_score(y_true=np.array([[1],[2],[3]]), y_pred=np.array([1,0,0]), average='micro')

0.3333333333333333

In [34]:
import numpy as np

a = np.array([1,2,3])
b = np.array([0,1, 3, 0, 0])

np.nonzero(~np.in1d(b, a))[0]

array([0, 3, 4])

In [38]:
b[np.array([0, 3, 4])]

array([0, 0, 0])

In [32]:
np.array(np.array([1,2,3]))

array([1, 2, 3])

In [12]:
net = torch.load('./Model_weights/rubert_50.pt')

In [13]:
from tqdm import tqdm
from sklearn.metrics import f1_score
import pandas as pd

testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)
net.eval()
model_answers=[]
real_answers=[]
with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        inputs = data['tokenized_phrases']

        with torch.no_grad():
            outputs_dict = net(inputs)
            pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'])
        
    f1 = f1_score(real_answers, model_answers, average='micro')
    print('%s micro'%f1)
    f1 = f1_score(real_answers, model_answers, average='macro')
    print('%s macro'%f1)
            


100%|██████████| 3787/3787 [00:41<00:00, 91.47batch/s] 


0.76234486400845 micro
0.2934589715874881 macro


---

In [14]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score



#Гиперпараметры
batch_size=16
epochs = 50
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

net.train()
initial_loss = None
best_f1=0
for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if USE_CUDA:
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scheduler.step()
                #scheduler.step()
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                scheduler.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'])
    f1 = f1_score(real_answers, model_answers, average='micro')
    if f1>best_f1:
        best_f1=f1
        torch.save(net,'./Model_weights/xlmrobertasag_best_cycliclr_%s.pt'%epoch)
    if epoch==9:
        torch.save(net,'./Model_weights/xlmrobertasag_cycliclr_%s.pt'%epoch)
    print(f1)
            
print('Finished Training')
torch.save(net,'./Model_weights/xlmrobertasag_cycliclr_%s.pt'%epoch)

100%|██████████| 3787/3787 [01:22<00:00, 45.94batch/s]


0.28201742804330604


100%|██████████| 3787/3787 [01:14<00:00, 50.50batch/s]


0.4415104304198574


100%|██████████| 3787/3787 [01:16<00:00, 49.60batch/s]


...

0.7219434908898863


100%|██████████| 3787/3787 [01:15<00:00, 50.03batch/s]


0.7222075521520993


100%|██████████| 3787/3787 [01:14<00:00, 50.98batch/s]


0.7280168999207816
Finished Training


<h3>Процесс обучения и рассчета точностей xlmrobertasag</h3>

In [14]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score

#Гиперпараметры
batch_size=16
epochs = 50
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

net.train()
initial_loss = None
best_f1=0
for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if device=='cuda':
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'])
    f1 = f1_score(real_answers, model_answers, average='micro')
    if f1>best_f1:
        best_f1=f1
        torch.save(net,'./Model_weights/xlmroberta_best_%s.pt'%epoch)
    if epoch==9:
        torch.save(net,'./Model_weights/xlmroberta_%s.pt'%epoch)
    print(f1)
            
print('Finished Training')
torch.save(net,'./Model_weights/xlmroberta_%s.pt'%epoch)

100%|██████████| 3787/3787 [01:16<00:00, 49.41batch/s]


0.2207552152099287


100%|██████████| 3787/3787 [01:21<00:00, 46.64batch/s]


0.39820438341695275


100%|██████████| 3787/3787 [01:17<00:00, 48.72batch/s]


...

0.7127013467124372


100%|██████████| 3787/3787 [01:15<00:00, 50.41batch/s]


0.7060998151571165


100%|██████████| 3787/3787 [01:16<00:00, 49.75batch/s]


0.7097966728280961
Finished Training


Инференс (подать фразу в сетку и посмотреть, какой концепт она выдаст)

Чтобы сетка выдавала отсутствие концепта на несвязанные с медрой фразы, раскомментируйте строчку с label_concepless_tensors

In [60]:
from transformers import AutoTokenizer

phrase = 'Disease'
tokenizer = AutoTokenizer.from_pretrained(transformer_model_path)
encoded_input = tokenizer([phrase], padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    outputs_dict = net(inputs)
    #outputs_dict.label_concepless_tensors(score_treshold = 6.1977e-05)
    pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]
    
print('phrase: %s'%phrase)
print('model output: %s'%CV.meddra_code_to_meddra_term[pred_meddra_code])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


phrase: Disease
model output: Myalgia


<h2>Сохранение и загрузка модели</h2>

сохранение

In [63]:
torch.save(net, './trained_model_weights.pt')
torch.save(optimizer.state_dict(), './trained_model_opt.pt')

загрузка

In [64]:
the_model = torch.load('./trained_model_weights.pt')